In [1]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
import gspread_dataframe as gd
import pygsheets
from datetime import date
import json

# Package to save datasets to be used in another script for next analysis
from sklearn import datasets

# enableing max columns for dataframes
pd.set_option('display.max_columns', None)

# Importing credentials for Strava's API
from Credentials import StravaCredentials

In [4]:
# Copy and paste this link in your browser and extract code -> #https://www.strava.com/oauth/authorize?client_id=99205&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
data = StravaCredentials.data

In [5]:
# Creating date variable
today = date.today().strftime('%B/%m/%Y')

In [6]:
# Getting refresh token
token = requests.post(url= 'https://www.strava.com/api/v3/oauth/token',data=data).json()

# Accessing the token json to get refresh token and access token
refresh_token = token['refresh_token']
access_token = token['access_token']
access_token

'a85e266930e3eb76ddd7a213ae6c5641dd1a0cdb'

In [ ]:
# Getting athlete info
athlete_url = 'https://www.strava.com/api/v3/athlete'
athlete = requests.get(athlete_url+'?access_token='+access_token).json()

# Covnerting json into dataframe and doing some data cleaning
df_athlete = pd.json_normalize(athlete)
df_athlete = df_athlete.T
df_athlete = df_athlete.reset_index()
df_athlete = df_athlete.rename(columns={0:'info'})
df_athlete

In [7]:
# Setting up url and page
# This API request gives us the list of activities. 
# The table lacks certain details that we will get from another API request
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = token['access_token']
# Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame()
while True:
    # get page of activities from Strava
    print('Getting page number:',page)
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
    print(f'Extraction of page {page} Complete')
    # if no results then exit loop
    if (not r):
        print('Extration Done')
        break
    r = pd.json_normalize(r)
    activities = activities.append(r)
    
    page += 1

clean_activities = activities[['id',
'name',
'distance',
'elapsed_time',
'total_elevation_gain',
'sport_type',
'start_date','achievement_count',
'athlete_count',
'start_latlng',
'end_latlng',
'average_speed',
'max_speed',
'average_temp',
'average_heartrate',
'max_heartrate',
'average_cadence',
'elev_high',
'elev_low']]

Getting page number: 1
Extraction of page 1 Complete
Getting page number: 2
Extraction of page 2 Complete
Getting page number: 3
Extraction of page 3 Complete
Getting page number: 4
Extraction of page 4 Complete
Getting page number: 5
Extraction of page 5 Complete
Extration Done


In [8]:
# Checkpoint so I don't have to run API get request
#activities_copy = pd.read_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Outputs\clean_activities.csv')

# Creating copy of activitieas dataframe and renaming some columns
general_activities_copy = clean_activities.copy()
general_activities_copy = general_activities_copy.rename(columns={'elapsed_time':'workout_time_sec','sport_type':'workout_type'})
general_activities_copy.head()

,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low
0,8683360176,Afternoon Workout,0.0,6429,0.0,Workout,2023-03-08T21:24:26Z,0,1,[],[],0.000,0.000,29.0,106.7,158.0,NaN,NaN,NaN
1,8677673116,Afternoon Workout,0.0,2910,0.0,Workout,2023-03-07T20:54:54Z,0,1,[],[],0.000,0.000,31.0,127.2,158.0,NaN,NaN,NaN
2,8672944578,Evening Walk,6289.2,6739,43.0,Walk,2023-03-07T00:28:09Z,0,1,"[30.38993197493255, -97.73823511786759]","[30.389002170413733, -97.73581366986036]",1.103,1.947,27.0,82.1,98.0,50.2,265.0,244.4
3,8672611987,Afternoon Ride,0.0,2875,0.0,Ride,2023-03-06T23:37:56Z,0,1,[],[],0.000,0.000,30.0,80.8,91.0,NaN,NaN,NaN
4,8655132144,Afternoon Workout,0.0,2255,0.0,Workout,2023-03-03T23:12:55Z,0,1,[],[],0.000,0.000,28.0,140.6,173.0,NaN,NaN,NaN


In [9]:
# Cleaning dataframe and converting units 
# Aprox calories burned during workout calculation
# Formual -> CB = T * (0.6309*H + 0.1988*W + 0.2017*A - 55.0969) / 4.184
# CB ->  is the number of calories burned
# T ->  is the duration of exercise in minutes
# H ->  is your average heart rate in beats per minute
# W -> is your weight in kilograms = 80
# A -> is your age in years
general_activities_copy['aprox_calories_burned'] = round((general_activities_copy['workout_time_sec']/60) * ((0.6309*general_activities_copy['average_heartrate']) + (0.1988*80) + (0.2017*26 - 55.0969)) / 4.184,0)

# from meters to kilometers
general_activities_copy[['distance']] = round(general_activities_copy['distance']/1000,2)

# from seconds to minutes
general_activities_copy['workout_time_min'] = round(general_activities_copy['workout_time_sec']/60,2)

# Fix start_date column into the correct format
general_activities_copy[["start_date"]] = pd.to_datetime(general_activities_copy['start_date']).dt.date

# Changing name of workout type => Workout
general_activities_copy['workout_type'] = general_activities_copy['workout_type'].replace({'Workout':'Functional-Cardio Workout'})

# Preview
print('Number of recorded workouts:',len(general_activities_copy))
general_activities_copy.head()

Number of recorded workouts: 641


,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low,aprox_calories_burned,workout_time_min
0,8683360176,Afternoon Workout,0.00,6429,0.0,Functional-Cardio Workout,2023-03-08,0,1,[],[],0.000,0.000,29.0,106.7,158.0,NaN,NaN,NaN,855.0,107.15
1,8677673116,Afternoon Workout,0.00,2910,0.0,Functional-Cardio Workout,2023-03-07,0,1,[],[],0.000,0.000,31.0,127.2,158.0,NaN,NaN,NaN,537.0,48.50
2,8672944578,Evening Walk,6.29,6739,43.0,Walk,2023-03-07,0,1,"[30.38993197493255, -97.73823511786759]","[30.389002170413733, -97.73581366986036]",1.103,1.947,27.0,82.1,98.0,50.2,265.0,244.4,479.0,112.32
3,8672611987,Afternoon Ride,0.00,2875,0.0,Ride,2023-03-06,0,1,[],[],0.000,0.000,30.0,80.8,91.0,NaN,NaN,NaN,195.0,47.92
4,8655132144,Afternoon Workout,0.00,2255,0.0,Functional-Cardio Workout,2023-03-03,0,1,[],[],0.000,0.000,28.0,140.6,173.0,NaN,NaN,NaN,492.0,37.58


In [10]:
# Creating additional dataframes for specific activities:
# Running type workouts
running_activities = general_activities_copy.loc[general_activities_copy['workout_type'].isin(['Run','TrailRun'])]

# Biking type workouts
biking_activities = general_activities_copy.loc[general_activities_copy['workout_type'].isin(['Ride','MountainBikeRide'])]

# Functional type workouts
functional_activities = general_activities_copy.loc[general_activities_copy['workout_type'].isin(['Functional-Cardio Workout'])]

In [11]:
# Variables for activities breakdown dataframe 
today_msg = f'Total workouts as of {today}'
total_workouts = len(general_activities_copy)

# Breakdown of workout types:
new = [today_msg,total_workouts]
activities_breakdown = general_activities_copy['workout_type'].value_counts().rename_axis('Sport').reset_index(name='Count')
activities_breakdown = activities_breakdown.append(pd.Series(new, index=['Sport','Count']), ignore_index=True)
activities_breakdown

,Sport,Count
0,Functional-Cardio Workout,480
1,Run,60
2,WeightTraining,29
3,Ride,17
4,Swim,16
5,Walk,14
6,MountainBikeRide,8
7,Yoga,5
8,TrailRun,3
9,AlpineSki,3


In [12]:

# Creating list of ids of these activities to use in the detailed workout api request 
running_id_list = list(running_activities['id'])
biking_id_list = list(biking_activities['id'])
functional_id_list = list(functional_activities['id'])

# General statistics
first_recorded_workout = min(general_activities_copy['start_date'])
most_recent_workout=max(general_activities_copy['start_date'])
average_workout_duration=round(general_activities_copy['workout_time_min'].mean(),2)
average_calories_burned_per_workout=round(general_activities_copy['aprox_calories_burned'].mean(),0)
average_distance_ran=round(running_activities['distance'].mean(),0)
average_biking_distance=round(biking_activities['distance'].mean(),0)

# Storing average calories for later analysis comparing calories from garmin, strava and my calculated value
%store average_calories_burned_per_workout

# Creating dataframe from general statistics variables
# Create the DataFrame
general_stats_df = pd.DataFrame({
    'First Workout':first_recorded_workout,
    'Most Recent Workout': most_recent_workout,
    'Average Workout Duration in Minutes':average_workout_duration,
    'Average Calories Burned Per Workout':average_calories_burned_per_workout,
    'Average Distance Ran in Kilometers':average_distance_ran,
    'Average Biking Distance in Kilometers':average_biking_distance
},index=['Info'])

# Transposing dataframe, setting new index and column
general_stats_df = general_stats_df.T
general_stats_df = general_stats_df.reset_index()
general_stats_df = general_stats_df.rename(columns={'index':'Info','Info':'Data'})
general_stats_df

Stored 'average_calories_burned_per_workout' (float)


,Info,Data
0,First Workout,2020-05-20
1,Most Recent Workout,2023-03-08
2,Average Workout Duration in Minutes,40.98
3,Average Calories Burned Per Workout,509
4,Average Distance Ran in Kilometers,7
5,Average Biking Distance in Kilometers,7


In [14]:
# General statistics - pending incorporate some of this numbers to the avobe graph
# Pending Analysis
general_activities_copy[['distance','workout_time_min','average_speed','max_speed','average_heartrate','average_temp','max_heartrate','average_cadence','aprox_calories_burned']].describe()

,distance,workout_time_min,average_speed,max_speed,average_heartrate,average_temp,max_heartrate,average_cadence,aprox_calories_burned
count,641.000000,641.000000,641.000000,641.000000,622.000000,612.000000,622.000000,70.000000,622.000000
mean,1.144041,40.980374,0.390398,0.924661,136.610129,29.357843,167.596463,70.574286,508.512862
std,3.349527,23.871215,0.991292,2.469820,16.712994,4.187237,18.135636,19.632978,275.277753
min,0.000000,2.580000,0.000000,0.000000,68.800000,9.000000,81.000000,26.100000,13.000000
25%,0.000000,29.950000,0.000000,0.000000,131.025000,27.000000,164.250000,51.575000,359.500000
50%,0.000000,37.580000,0.000000,0.000000,140.350000,30.000000,173.000000,83.000000,483.000000
75%,0.000000,47.250000,0.000000,0.000000,146.575000,33.000000,178.000000,84.775000,598.500000
max,34.470000,376.470000,6.949000,18.600000,170.500000,37.000000,195.000000,89.000000,2707.000000


In [15]:
# Get detailed view of workouts function:
# This function will get the data for each workout, if it reaches the API request limit it will stop the process
# The API rate limit allows us to do 100 requests for each 15 mintues. To prvent passing this limit we only grab -
# - the most recent 100 workouts from each list.
def GetWorkoutData(workout_list):
    workout_info = []
    workout_num = 1
    if len(workout_list)>100:
        print('This workout list is too large, reducing to the 100 most recent workouts.')
        workout_list = workout_list[:100]
        for i in workout_list:
            print('Extracting workout:', workout_num)
            req = requests.get(url = f'https://www.strava.com/api/v3/activities/{i}?access_token='+access_token)
            if req.status_code == 200:
                req = req.json()
                workout_info.append(req)
                workout_num += 1
            else:
                print('API Rate limit exceeded, stopping extraction')
                break
    else:
        for i in workout_list:
            print('Extracting workout:',workout_num)
            req = requests.get(url = f'https://www.strava.com/api/v3/activities/{i}?access_token='+access_token)
            if req.status_code == 200:
                req = req.json()
                workout_info.append(req)
                workout_num += 1
            else:
                print('API Rate limit exceeded, stopping extraction')
                break


    return workout_info 

In [ ]:
# Test code
#pd.json_normalize(requests.get(url = f'https://www.strava.com/api/v3/activities/8611884034?access_token='+access_token).json())

In [24]:
# Using the function to get a json of the detailed data for functional, running and biking workouts
# Make sure to run one at a time every 15 minutes to not reach API rate limit
functional_workouts = GetWorkoutData(functional_id_list)
#running_workouts = GetWorkoutData(running_id_list[0:1])
biking_workouts = GetWorkoutData(biking_id_list[0:2])

# saving a copy for further analysis
#functional_workouts_json = functional_workouts

Extracting workout: 1
Extracting workout: 2


In [25]:
test = pd.json_normalize(biking_workouts, 'laps')
test

,id,resource_state,name,elapsed_time,moving_time,start_date,start_date_local,distance,start_index,end_index,total_elevation_gain,average_speed,max_speed,device_watts,average_heartrate,max_heartrate,lap_index,split,activity.id,activity.resource_state,athlete.id,athlete.resource_state,average_watts
0,29352460198,2,Lap 1,2874,2702,2023-03-06T23:37:56Z,2023-03-06T17:37:56Z,0.00,0,966,0.0,0.00,0.000,False,80.8,91.0,1,1,8672611987,1,58832723,1,NaN
1,29219602604,2,Lap 1,1870,1807,2023-02-28T23:21:36Z,2023-02-28T17:21:36Z,8046.72,0,1246,118.4,4.45,9.694,False,121.1,156.0,1,1,8639260773,1,58832723,1,92.0
2,29219602630,2,Lap 2,1328,1328,2023-02-28T23:52:49Z,2023-02-28T17:52:49Z,4667.97,1247,2110,66.8,3.52,6.950,False,138.4,157.0,2,2,8639260773,1,58832723,1,81.3


# Laps dataframe ideas:
#### merge the functional workout dataframe to the number of laps per workout. Do this by using a group by (?)
####

In [18]:
# Getting laps from the functional workouts dataframe
functional_workouts_laps = pd.json_normalize(functional_workouts,'laps')
functional_workouts_laps = functional_workouts_laps.rename(columns={'activity.id':'activity_id'})
functional_workouts_laps

,id,resource_state,name,elapsed_time,moving_time,start_date,start_date_local,distance,start_index,end_index,total_elevation_gain,average_speed,max_speed,device_watts,average_heartrate,max_heartrate,lap_index,split,activity_id,activity.resource_state,athlete.id,athlete.resource_state
0,29395205822,2,Lap 1,3968,880,2023-03-08T21:24:26Z,2023-03-08T15:24:26Z,0.0,0,381,0,0.0,0.0,False,90.3,129.0,1,1,8683360176,1,58832723,1
1,29395205824,2,Lap 2,451,325,2023-03-08T22:30:37Z,2023-03-08T16:30:37Z,0.0,382,499,0,0.0,0.0,False,120.2,133.0,2,2,8683360176,1,58832723,1
2,29395205826,2,Lap 3,407,407,2023-03-08T22:38:05Z,2023-03-08T16:38:05Z,0.0,500,645,0,0.0,0.0,False,132.6,154.0,3,3,8683360176,1,58832723,1
3,29395205828,2,Lap 4,431,431,2023-03-08T22:44:53Z,2023-03-08T16:44:53Z,0.0,646,815,0,0.0,0.0,False,134.9,155.0,4,4,8683360176,1,58832723,1
4,29395205830,2,Lap 5,526,526,2023-03-08T22:52:05Z,2023-03-08T16:52:05Z,0.0,816,1024,0,0.0,0.0,False,137.9,156.0,5,5,8683360176,1,58832723,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,26987296296,2,Lap 2,4,4,2022-11-03T01:16:36Z,2022-11-02T20:16:36Z,0.0,385,388,0,0.0,0.0,False,131.8,133.0,2,2,8068964116,1,58832723,1
398,26987296297,2,Lap 3,530,530,2022-11-03T01:16:43Z,2022-11-02T20:16:43Z,0.0,389,620,0,0.0,0.0,False,145.3,161.0,3,3,8068964116,1,58832723,1
399,26987296298,2,Lap 4,550,550,2022-11-03T01:25:36Z,2022-11-02T20:25:36Z,0.0,621,900,0,0.0,0.0,False,147.5,172.0,4,4,8068964116,1,58832723,1
400,26987296300,2,Lap 5,196,196,2022-11-03T01:34:43Z,2022-11-02T20:34:43Z,0.0,901,970,0,0.0,0.0,False,134.8,159.0,5,5,8068964116,1,58832723,1


In [38]:
def CleanWorkoutDF(df):
    df[['distance']] = round(df['distance']/1000,2)
    df['workout_time_min'] = round(df['elapsed_time']/60,2)
    df[["start_date"]] = pd.to_datetime(df['start_date']).dt.date
    return df[['id',
               'name',
               'start_date',
               'sport_type',
               'distance',
               'workout_time_min',
               'calories',
               'total_elevation_gain',
               'start_latlng',
               'end_latlng',
               'average_speed',
               'max_speed',
               'average_temp',
               'average_heartrate',
               'max_heartrate']]

In [39]:
# Pending -> add average number of laps
def DescribeWorkoutDF(df):
    average_workout_duration=round(df['workout_time_min'].mean(),2)
    average_calories_burned_per_workout=round(df['calories'].mean(),0)
    average_distance=round(df['distance'].mean(),0)

    # Creating dataframe from general statistics variables
    # Create the DataFrame
    general_stats_df = pd.DataFrame({
        'Average Workout Duration in Minutes':average_workout_duration,
        'Average Calories Burned Per Workout':average_calories_burned_per_workout,
        'Average Distance in Kilometers':average_distance
    },index=['Info'])

    # Transposing dataframe, setting new index and column
    general_stats_df = general_stats_df.T
    general_stats_df = general_stats_df.reset_index()
    general_stats_df = general_stats_df.rename(columns={'index':'Info','Info':'Data'})
    general_stats_df

,id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate
0,8683360176,Afternoon Workout,2023-03-08,Workout,0.0,107.15,448.0,0,[],[],0.0,0.0,29,106.7,158.0
1,8677673116,Afternoon Workout,2023-03-07,Workout,0.0,48.50,435.0,0,[],[],0.0,0.0,31,127.2,158.0
2,8655132144,Afternoon Workout,2023-03-03,Workout,0.0,37.58,400.0,0,[],[],0.0,0.0,28,140.6,173.0
3,8650266138,Afternoon Workout,2023-03-02,Workout,0.0,57.88,561.0,0,[],[],0.0,0.0,29,146.1,181.0
4,8644587607,Afternoon Workout,2023-03-01,Workout,0.0,36.90,367.0,0,[],[],0.0,0.0,30,148.2,177.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,8083176906,Lunch Workout,2022-11-07,Workout,0.0,21.15,255.0,0,[],[],0.0,0.0,31,150.6,174.0
75,8079471189,Lunch Workout,2022-11-06,Workout,0.0,61.37,499.0,0,[],[],0.0,0.0,31,133.2,165.0
76,8068964123,Afternoon Workout,2022-11-04,Workout,0.0,54.23,447.0,0,[],[],0.0,0.0,32,131.6,173.0
77,8068964116,Evening Workout,2022-11-03,Workout,0.0,34.80,370.0,0,[],[],0.0,0.0,29,140.4,172.0


In [36]:
biking_workouts_df = pd.json_normalize(biking_workouts)
CleanWorkoutdf(biking_workouts_df)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete.id,athlete.resource_state,map.id,map.polyline,map.resource_state,map.summary_polyline,photos.primary,photos.count,average_watts,kilojoules,device_watts,elev_high,elev_low,splits_metric,splits_standard,workout_time_min
0,3,Afternoon Ride,0.00,2875,2875,0.0,Ride,Ride,None,8672611987,2023-03-06,2023-03-06T17:37:56Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.000,0.000,30,True,80.8,91.0,False,True,9309518497,9309518497,garmin_ping_263187668145,False,0,0,False,None,133.0,None,None,[],"[{'id': 29352460198, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,386bdbe15457f47a57cd609e7a1577a61e97b58a,[],58832723,1,a8672611987,,3,,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.92
1,3,Afternoon Mountain Bike Ride,12.71,2955,3200,169.0,Ride,MountainBikeRide,None,8639260773,2023-02-28,2023-02-28T17:21:36Z,(GMT-06:00) America/Chicago,-21600.0,None,None,United States,11,0,0,1,0,False,False,False,False,everyone,False,None,"[30.401550130918622, -97.6859537512064]","[30.401377798989415, -97.68642079085112]",4.303,9.694,28,True,128.1,157.0,False,True,9273997646,9273997646,garmin_ping_262248927036,False,5,0,False,None,457.0,None,None,"[{'id': 3065805878067921098, 'resource_state':...","[{'id': 29219602604, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,cb814c4891ba870881452a8a714bf41f2ced2bd6,[],58832723,1,a8639260773,uxpxDfhfsQr@lCHx@Vl@p@bAb@v@LPRHV@VJHHPRv@hARb...,3,ctpxDprfsQf@x@j@J`@ThA|AVn@Ft@Aj@Md@y@x@eAn@wA...,None,0,87.7,259.0,False,270.6,244.2,"[{'distance': 1002.2, 'elapsed_time': 298, 'el...","[{'distance': 1614.2, 'elapsed_time': 397, 'el...",53.33


In [30]:
functional_workouts_df = pd.json_normalize(functional_workouts)
functional_workouts_df.head(1)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete.id,athlete.resource_state,map.id,map.polyline,map.resource_state,map.summary_polyline,photos.primary,photos.count,photos.primary.unique_id,photos.primary.urls.600,photos.primary.urls.100,photos.primary.source,photos.primary.media_type,photos.use_primary_photo
0,3,Afternoon Workout,0.0,6429,6429,0,Workout,Workout,8683360176,2023-03-08T21:24:26Z,2023-03-08T15:24:26Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.0,0.0,29,True,106.7,158.0,False,True,9320764358,9320764358,garmin_ping_263487797017,False,0,0,False,None,448.0,NaN,None,[],"[{'id': 29395205822, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,3032bf7658c9b6c1080db066eefcba71e4a40d01,[],58832723,1,a8683360176,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
DescribeWorkoutdf(functional_workouts_df)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete.id,athlete.resource_state,map.id,map.polyline,map.resource_state,map.summary_polyline,photos.primary,photos.count,photos.primary.unique_id,photos.primary.urls.600,photos.primary.urls.100,photos.primary.source,photos.primary.media_type,photos.use_primary_photo,workout_time_min
0,3,Afternoon Workout,0.0,6429,6429,0,Workout,Workout,8683360176,2023-03-08,2023-03-08T15:24:26Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.0,0.0,29,True,106.7,158.0,False,True,9320764358,9320764358,garmin_ping_263487797017,False,0,0,False,None,448.0,NaN,None,[],"[{'id': 29395205822, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,3032bf7658c9b6c1080db066eefcba71e4a40d01,[],58832723,1,a8683360176,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,107.15
1,3,Afternoon Workout,0.0,2910,2910,0,Workout,Workout,8677673116,2023-03-07,2023-03-07T14:54:54Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.0,0.0,31,True,127.2,158.0,False,True,9314816041,9314816041,garmin_ping_263327081345,False,0,0,False,None,435.0,NaN,None,[],"[{'id': 29372873332, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,1c0d1a07d81912a180e2280ff41d53304e7464fe,[],58832723,1,a8677673116,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,48.50
2,3,Afternoon Workout,0.0,2255,2255,0,Workout,Workout,8655132144,2023-03-03,2023-03-03T17:12:55Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.0,0.0,28,True,140.6,173.0,False,True,9290602782,9290602782,garmin_ping_262700350793,False,0,0,False,None,400.0,NaN,None,[],"[{'id': 29281313914, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,33b8980ac1b25e6dec0b3467f26683ea4702e985,[],58832723,1,a8655132144,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,37.58
3,3,Afternoon Workout,0.0,3473,3473,0,Workout,Workout,8650266138,2023-03-02,2023-03-02T17:01:05Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.0,0.0,29,True,146.1,181.0,False,True,9285506253,9285506253,garmin_ping_262554189934,False,0,0,False,None,561.0,NaN,None,[],"[{'id': 29262734014, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,d58621cc97eabde8d539a0c00a854f5a35a482ef,[],58832723,1,a8650266138,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,57.88
4,3,Afternoon Workout,0.0,2214,2214,0,Workout,Workout,8644587607,2023-03-01,2023-03-01T16:28:22Z,(GMT-06:00) America/Bahia_Banderas,-21600.0,None,None,United States,0,0,0,1,0,True,False,False,False,everyone,False,None,[],[],0.0,0.0,30,True,148.2,177.0,False,True,9279574758,9279574758,garmin_ping_262395819775,False,0,0,False,None,367.0,NaN,None,[],"[{'id': 29240677379, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin fēnix 6 Pro,82c752b1a34172f03c4cb66b159a24a7aa0370b8,[],58832723,1,a8644587607,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,36.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
functional_workouts_laps = functional_workouts_laps[functional_workouts_laps.activity_id != 8611884034]
functional_workouts_laps['activity_id'].value_counts()

In [ ]:
functional_workouts_df = pd.json_normalize(functional_workouts)
functional_workouts_df.head()

# Checkpoint -> connect to google drive api -> then connect to google looker

#### article in stackoverflow to set up function to upload data to googlesheet:
#### https://stackoverflow.com/questions/62917910/how-can-i-export-pandas-dataframe-to-google-sheets-using-python

In [ ]:
# This function uses gspread and pygsheets modules to upload data to google sheets
def WriteToGsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1
    
# Setting up parameters for write_to_gsheet function
service_file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'
   

In [ ]:
general_activities_copy

In [ ]:
# Uploading functional workouts to google sheets:
functional_workouts_sheet = 'specific_data' 
WriteToGsheet(service_file_path,spreadsheet_id,functional_workouts_sheet,functional_workouts_df)

In [ ]:
# Uploading general stats dataframe to google sheets:
general_stats_sheet = 'General_Statistics' 
WriteToGsheet(service_file_path,spreadsheet_id,general_stats_sheet,general_stats_df)

In [ ]:
# Uploading activities breakdown to google sheets:
activities_breakdown_sheet = 'Activities_Breakdown'
WriteToGsheet(service_file_path,spreadsheet_id,activities_breakdown_sheet,activities_breakdown)

In [ ]:
#creating checkpoint !
# In this example, the my_list variable represents the list that we want to save as a JSON file. We use the open() function to open a new file called "my_list.json" in write mode, and then use the json.dump() method to write the list to the file in JSON format.
with open('functional_workouts_json.json','w') as f:
    json.dump(functional_workouts_json,f)
# The with statement is used here to automatically close the file when we're done writing to it. The "w" parameter in the open() function specifies that we're opening the file in write mode. If the file already exists, its contents will be overwritten. If it doesn't exist, a new file will be created.
# f = open(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Outputs\functional_workouts_json.json')
# functional_workouts_json = json.load(f)